In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import codecs
# Task: 
#  1) determine the five most common journals and the total articles for each. 
#  2) Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . 

In [2]:
wellcome1 = pd.read_csv('clean.csv',  encoding='latin-1') #figured it out but i dont understand what this means 
wellcome1

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [3]:
# Before Cleaning 
#Lets take a look at the Journal Titles, and get an idea of how messy the data is 
(wellcome1.groupby('Journal title').count())
#How do we determine the five most common journals? currently 984 different journals 

# There is a need to fix the case 
#capitalization differences, space differences, remove puntuation 
wellcome1['Journal title'].value_counts()[:5]



PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
Name: Journal title, dtype: int64

In [5]:
# Removing popular duplicates and making sure all the same cases to prevent misplaced counts
wellcome1['JOURNAL TITLE'] = wellcome1['Journal titele'].replace(['PLOSONE', 'PLOS  ONE', 'PLOS 1', 'PLOS', 'PLoS ONE','PLoS One'], 'PLOS ONE')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['ACTA D', 'ACTA CRYSTALLOGRAPHICA SECTION D', 'ACTA CRYSTALLOGRAPHY D', 'ACTA CRYSTALLOGRAPHICA, SECTION D', 'ACTA CRYSTALLOGRAPHICA SECTION D,  BIOLOGICAL CRYSTALLOGRAPHY'], 'ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['AMERICAN JNL EPIDEMIOLOGY'], 'AMERICAN JOURNAL OF EPIDEMIOLOGY')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['AMERICAN JOURNAL OF MEDICAL GENETICS PART A'], 'AMERICAN JOURNAL OF MEDICAL GENETICS')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['ANTIMICROBIAL AGENTS AND CHEMOTHERAPY', 'ANTIMICROBIAL AGFENTS AND CHEMOTHERAPY'], 'ANTIMICROBIAL AGENTS & CHEMOTHERAPY')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['ANGEWANDE CHEMIE', 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION','ANGEW CHEMS INT ED' ], 'ANGEWANDTE CHEMIE')
wellcome1['JOURNAL TITLE'] = wellcome1['Journal title'].replace(['BEHAVIOUR RESEARCH AND THERAPY'], 'BEHAVIOR RESEARCH & THERAPY')
wellcome1['Journal title']= wellcome1['Journal title'].str.upper()

In [6]:
# the five most common journals 
wellcome1['Journal title'].value_counts()[:5]

PLOS ONE                           190
JOURNAL OF BIOLOGICAL CHEMISTRY     53
NEUROIMAGE                          29
PLOS PATHOGENS                      24
PLOS GENETICS                       24
Name: Journal title, dtype: int64

In [7]:
# Removing the £ symbol from price so
wellcome1['journal_price'] = wellcome1['COST (£) charged to Wellcome (inc VAT when charged)'].str.lstrip('£')
del wellcome1['COST (£) charged to Wellcome (inc VAT when charged)']
# The price is reported as a string and cannot be acted upon, now we must conver to an integer 
print(type('journal_price'))

<class 'str'>


In [8]:
# converting prices from strings to integers 
wellcome1['journal_prices'] = pd.to_numeric(wellcome1.journal_price, errors='coerce')
wellcome1.journal_prices.describe()

count      2114.000000
mean      24206.441892
std      147300.992149
min           0.000000
25%        1280.000000
50%        1889.950000
75%        2322.315000
max      999999.000000
Name: journal_prices, dtype: float64

In [11]:
# Listing the average price of each journal 
wellcome1.groupby('Journal title').describe()

journal_prices  \
                                                            count   
Journal title                                                       
ACADEMY OF NUTRITION AND DIETETICS                            1.0   
ACS CHEMICAL BIOLOGY                                          5.0   
ACS CHEMICAL NEUROSCIENCE                                     1.0   
ACS NANO                                                      2.0   
ACTA CRYSTALLOGRAPHICA SECTION D,  BIOLOGICAL C...            1.0   
ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CR...            1.0   
ACTA CRYSTALLOGRAPHICA SECTION F: STRUCTURAL BI...            2.0   
ACTA CRYSTALLOGRAPHICA, SECTION D                             1.0   
ACTA CRYSTALLOGRAPHY D                                        1.0   
ACTA D                                                        1.0   
ACTA DERMATO VENEREOLOGICA                                    1.0   
ACTA DIABETOLOGICA                                            1.0   
ACTA F                                                        1.0   
ACTA NEUROPATHOL                                              1.0   
ACTA NEUROPATHOLOGICA                                         3.0   
ACTA OPTHALMOLOGICA                                           1.0   
ACTA PHYSIOL                                                  1.0   
ADDICTION                                                     2.0   
ADVANCES IN EXPERIMENTAL MEDICINE AND BIOLOGY                 4.0   
AGE                                                           2.0   
AGE AND AGEING                                                1.0   
AGEING & SOCIETY                                              1.0   
AGING CELL                                                    1.0   
AIDS                                                          3.0   
AIDS BEHAV                                                    1.0   
AIDS CARE                                                     4.0   
AIDS JOURNAL                                                  1.0   
AIDS RESEARCH AND THERAPY                                     1.0   
AIDS UK                                                       2.0   
ALCOHOL AND ALCOHOLISM                                        1.0   
...                                                           ...   
TRAFFIC                                                       6.0   
TRANSACTIONS OF THE ROYAL SOCIETY OF TROPICAL M...            1.0   
TRANSLATIONAL PSYCHIATRY                                      2.0   
TRENDS IN COGNITIVE SCIENCE                                   1.0   
TRENDS IN GENETICS                                            2.0   
TRENDS IN MICROBIOLOGY                                        1.0   
TRENDS IN MOLECULAR MEDICINE                                  1.0   
TRENDS IN NEUROSCIENCE                                        1.0   
TRENDS IN NEUROSCIENCES                                       2.0   
TRENDS IN PARASITOLOGY                                        4.0   
TRIALS                                                        4.0   
TRIALS                                                        1.0   
TROP MED INT HEALTH                                           1.0   
TROPICAL ANIMAL HEALTH & PRODUCTION                           1.0   
TROPICAL MEDICINE AND INTERNATIONAL HEALTH                    6.0   
TUBERCULOSIS                                                  2.0   
URBAN HISTORY                                                 1.0   
UROLITHIASIS                                                  1.0   
VACCINE                                                       5.0   
VASCULAR PHARMACOLOGY                                         1.0   
VETERINARY MICROBIOLOGY                                       1.0   
VETERINARY PARASITOLOGY                                       2.0   
VETERINARY RECORD                                             1.0   
VETERINARY RESEARCH                                           1.0   
VIROLOGY                                         